In [3]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install langdetect
!{sys.executable} -m pip install tqdm
!{sys.executable} -m pip install gensim
!{sys.executable} -m pip install stop_words

  Running setup.py bdist_wheel for smart-open: started
  Running setup.py bdist_wheel for smart-open: finished with status 'done'
  Stored in directory: C:\Users\mlava\AppData\Local\pip\Cache\wheels\36\48\35\97efc2bd1b233627131c9a936c9de23681846db707b907d353
  Running setup.py bdist_wheel for bz2file: started
  Running setup.py bdist_wheel for bz2file: finished with status 'done'
  Stored in directory: C:\Users\mlava\AppData\Local\pip\Cache\wheels\31\9c\20\996d65ca104cbca940b1b053299b68459391c01c774d073126
Successfully built smart-open bz2file
  Running setup.py bdist_wheel for stop-words: started
  Running setup.py bdist_wheel for stop-words: finished with status 'done'
  Stored in directory: C:\Users\mlava\AppData\Local\pip\Cache\wheels\22\74\80\77275c2f9f2f1d9841b51e169a38985640a10fbd2711d10791
Successfully built stop-words


In [1]:
import numpy as np
import pandas as pd
import nltk
from tqdm import tqdm, tqdm_notebook # progress bars in Jupyter
from time import time # measure the computation time of a python code
import langdetect # detect the
import stop_words # handles stop words in many languages without having to rebuild them everytime
import gensim # for solid LDA implementation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

C:\Users\mlava\Anaconda3\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [10]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

corpora wordnet, stopwords
models punkt

File > change directory > C:\nltk_data pour une installation centrale

puis fermer fenetre

In [2]:
data = pd.read_csv('data_scraping_V2.csv', engine='python')
print(data.shape)
data.head()

(46806, 2)


,source,text
0,amazon,Love my S8! Awesome screen and takes great pic...
1,reddit,"I mean, while I don't think that's an especial..."
2,bestbuy,"It would appear that this ""open box"" like new ..."
3,youtube,During all ur review u just talked about getti...
4,youtube,I really want to trade my 7 plus for the S8 ?


data = pd.read_csv('data_scraping.csv', sep = ',', engine = 'python')

special character removal
tokenization monogram, bigram, trigram
stop words removal
stemming/ lemmatizationTfidfVectorizer(max_df = 0.95, min_df = 2, max_features = n_features, stop_words = 'english')

In [3]:
## on ne garde que les commentaires en Anglais

from langdetect.lang_detect_exception import LangDetectException

data['lang'] = '' # on ajoute une colonne langue
for index in range(len(data.index)):
    try:
        data.loc[index, 'lang'] = langdetect.detect(data.loc[index,'text'])
    except LangDetectException:
        data.loc[index, 'lang'] = np.nan

In [4]:
data['lang'].head()
print(data.shape)

text = data[data['lang'] == 'en' ].reset_index().text
text = text.replace(r'\n',' ', regex=True)
text = text.replace('\xa0','', regex = True)
text = text.replace('  ', ' ', regex = True)
print(text.shape)

(46806, 3)
(42125,)


In [5]:
for i in range(len(text)):
    text[i]=str(text[i]).lower()

text.head()

0    love my s8! awesome screen and takes great pic...
1    i mean, while i don't think that's an especial...
2    it would appear that this "open box" like new ...
3    during all ur review u just talked about getti...
4        i really want to trade my 7 plus for the s8 ?
Name: text, dtype: object

In [9]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

tokens = text.apply(nltk.word_tokenize,1)

for i in range(len(tokens)):
    tokens[i] = [word for word in tokens[i] if word.isalpha()]

stop_words = set(stopwords.words('english'))
    
for i in range(len(tokens)):
    tokens[i] = [w for w in tokens[i] if not w in stop_words]

tokens

0        [love, awesome, screen, takes, great, pictures...
1        [mean, think, especially, healthy, way, life, ...
2        [would, appear, open, box, like, new, samsung,...
3        [ur, review, u, talked, getting, used, thing, ...
4                              [really, want, trade, plus]
5        [sold, iphone, x, camera, performance, indoors...
6        [though, iphones, beginning, feel, like, next,...
7                  [love, samsung, edge, sleek, easy, use]
8        [fast, smooth, silk, internet, browsing, best,...
9        [confirmed, google, pixel, samsung, galaxy, no...
10       [apple, charger, ipads, charges, little, bit, ...
11                             [thankfully, happened, end]
12       [long, time, ios, user, switched, take, advant...
13                                 [dammit, ca, see, nose]
14                  [easy, use, fastest, phone, thus, far]
15       [third, iphone, great, new, features, user, fr...
16       [love, new, phone, took, bit, get, use, sensit.

In [52]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

stemmed = tokens.copy()
for i in range(len(stemmed)):
    stemmed[i] = [porter.stem(word) for word in stemmed[i]]
    
lemmatized = tokens.copy()

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
for i in range(len(lemmatized)):
    lemmatized[i] = [wordnet_lemmatizer.lemmatize(word) for word in lemmatized[i]]

lemmatized

0        [love, awesome, screen, take, great, picture, ...
1        [mean, think, especially, healthy, way, life, ...
2        [would, appear, open, box, like, new, samsung,...
3        [ur, review, u, talked, getting, used, thing, ...
4                              [really, want, trade, plus]
5        [sold, iphone, x, camera, performance, indoors...
6        [though, iphones, beginning, feel, like, next,...
7                  [love, samsung, edge, sleek, easy, use]
8        [fast, smooth, silk, internet, browsing, best,...
9        [confirmed, google, pixel, samsung, galaxy, no...
10       [apple, charger, ipads, charge, little, bit, s...
11                             [thankfully, happened, end]
12       [long, time, io, user, switched, take, advanta...
13                                 [dammit, ca, see, nose]
14                  [easy, use, fastest, phone, thus, far]
15       [third, iphone, great, new, feature, user, fri...
16       [love, new, phone, took, bit, get, use, sensit.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

# set the parameters for automatic selection of words
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features = 70, stop_words='english') 

In [11]:
TermDocument = tfidf_vectorizer.fit_transform(text.tolist()) # feed with the comments

In [12]:
# print the Term Document matrix
# columns are words, rows are documents (articles)
pd.DataFrame(TermDocument.todense()).replace(0, '')

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,,,,,0.575817,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,0.471107,,
2,,,,,,,0.40622,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,0.599947,,,,,,,
4,,,,,,,,,,,...,,,,,,,0.582239,,,
5,,,,,,,,,,,...,,,,,,,,,,
6,,,,,,,,0.661392,,,...,,,,,,,,,,
7,,,,,,,,,,,...,,0.44549,,,,,,,,
8,,,,,,,0.594918,,,,...,,,,,,,,,,
9,,,,,,,,,,,...,,,,,,,,,,


In [13]:
# the NMF decomposes this Term Document matrix into the product of 2 smaller matrices: W and H
n_dimensions = 10 # This can also be interpreted as topics in this case. This is the "beauty" of NMF. 10 is arbitrary
nmf = NMF(n_components=n_dimensions, random_state=42, alpha=.1, l1_ratio=.5).fit(TermDocument) 
# fixing the parameter "random_state=42" will result in having the same result if you re-run the algorithm again. 42 is arbitrary

In [15]:
# Term Document matrix = W x H, 
# W maps documents (articles) into new dimensions (in the case of NMF, we can interpret these as topics)
# H maps words to new dimensions
W = nmf.fit_transform(TermDocument)
H = nmf.components_

In [16]:
# print W
pd.DataFrame(W).replace(0, '')

,0,1,2,3,4,5,6,7,8,9
0,,,0.0213435,,0.0229786,,0.0260707,0.0273009,,
1,0.000330671,0.000149728,0.00149716,0.0023669,0.00144126,0.00917077,,0.00249201,0.00824115,0.00116957
2,,,0.0270659,0.0016772,,0.0252277,0.00755407,,,0.0448708
3,0.000376359,0.000307856,,,,,0.00121821,0.000846799,0.0389147,
4,0.000568468,0.00291438,0.0248267,,,0.00731778,0.000672147,0.00436582,0.00164204,
5,,0.0377349,,,0.0160157,0.000272623,0.00409239,0.000856959,,0.00112135
6,0.00765406,0.0128608,0.00354175,,0.0177318,0.0175847,0.00417213,0.00448848,,
7,0.000818816,,0.0395011,,0.00287185,,0.0296577,,0.000319449,
8,0.00025194,0.0260991,,,0.00233702,,,0.000392045,,0.0448402
9,,0.0206395,0.0360308,,,,,,,


In [17]:
# print H
pd.DataFrame(H).replace(0, '')

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,0.612681,0.230609,,0.208567,0.595976,0.212673,,0.417541,0.165468,0.467493,...,0.294809,0.737901,0.294145,0.338524,0.333295,,0.273298,0.277943,0.225904,0.228737
1,0.121128,0.275807,,0.0742849,0.0160521,0.2277,,0.557130,0.0896277,0.163016,...,0.496488,0.00802487,0.135818,0.189644,0.221402,0.249521,0.187317,0.132144,0.0768291,0.048828
2,0.00364691,1.01766,,0.108198,0.00463977,0.868407,,0.971460,0.0615857,0.191581,...,0.233220,0.323466,0.152053,0.277688,0.247611,0.217608,0.178664,0.171496,0.0938638,
3,,0.686498,12.5366,0.0603305,,0.375533,,0.119846,0.0605095,0.080994,...,0.000388,0.0877712,0.151614,0.0821464,0.142449,,0.128078,0.213469,0.105927,0.0534238
4,0.380608,,,0.20902,0.549561,1.38451,,0.414066,0.0778768,0.120731,...,0.513802,1.15812,0.151838,,0.029631,0.269508,,0.057718,0.165856,1.13486
5,,0.443041,,0.385093,,0.280106,,1.008458,0.282051,0.092857,...,0.122033,0.822257,0.438789,0.358833,0.389527,0.401142,0.328118,0.349134,0.380314,0.373178
6,0.859254,,,0.0592816,0.554541,,,0.011015,0.0817221,0.096238,...,0.215030,0.784755,0.312072,,0.011366,,,0.007016,0.0291454,0.0669387
7,0.719667,,,0.372715,0.510352,1.41291,,0.637432,0.870666,0.060624,...,0.255169,0.241674,0.406556,0.229978,0.282995,0.205878,0.115156,0.206479,0.199617,0.0880165
8,0.00102292,0.269869,,0.252245,,0.0933564,,0.236137,0.152266,0.172960,...,0.087581,0.709893,0.394028,0.28846,0.389306,0.317063,0.587731,0.445701,0.403221,0.309582
9,0.249336,0.114834,,,0.136795,,10.725,0.059203,,0.128064,...,0.018467,,0.137683,,0.734801,0.00843686,0.08292,0.069994,,0.0170851


In [18]:
# If the NMF worked well, the Term Document matrix should be similar to the product of the W and H
pd.DataFrame(TermDocument.toarray() - np.matmul(W, H)).replace(0, '')

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,-0.0508726,-0.0217204,,-0.0188333,0.534699,-0.0889228,,-0.0479386,-0.0290045,-0.0110273,...,-0.0293565,-0.060573,-0.0259697,-0.0122054,-0.0139881,-0.0164581,-0.00695719,-0.0108066,-0.0120241,-0.0302255
1,-0.00286821,-0.00968739,-0.0296729,-0.00722533,-0.00243027,-0.011148,-0.0125436,-0.0154085,-0.00642698,-0.00340934,...,-0.00376082,-0.0165997,-0.00936839,-0.00699164,-0.00923916,-0.00756624,-0.00892567,0.462678,-0.00802472,-0.00805798
2,-0.0177775,-0.0450248,-0.0210264,-0.0131925,-0.0104527,-0.0312004,-0.075021,-0.0546751,-0.00950118,-0.014137,...,-0.0118445,-0.0355738,-0.0239747,-0.0167062,-0.0498245,-0.0163882,-0.0170488,-0.0170012,-0.0125328,-0.0107763
3,-0.00196385,-0.0106736,,-0.0103052,-0.00133695,-0.00497953,,-0.0100711,-0.00685211,-0.0071254,...,-0.00415001,-0.0290661,0.583736,-0.0116059,-0.0155968,-0.0125896,-0.0231294,-0.017673,-0.0160044,-0.0123045
4,-0.004513,-0.0298853,,-0.00792054,-0.0031016,-0.0307158,,-0.036537,-0.00805436,-0.00679005,...,-0.0097,-0.0172388,-0.0101807,-0.0117428,-0.011715,-0.0104846,0.573233,-0.00899372,-0.0070189,-0.00394077
5,-0.0150792,-0.0106571,,-0.00681773,-0.0122675,-0.0320533,-0.0120265,-0.0285875,-0.0057868,-0.0086997,...,-0.0281165,-0.0224938,-0.00945643,-0.00745111,-0.0100484,-0.0140273,-0.0073495,-0.00629018,-0.00594947,-0.0204883
6,-0.0198242,-0.0167072,,-0.0153333,-0.0191335,-0.0434492,,0.619607,-0.0132269,-0.0108005,...,-0.0227667,-0.0462504,-0.0180718,-0.0133558,-0.0149682,-0.0167366,-0.0114204,-0.0125532,-0.0136958,-0.0297383
7,-0.0272226,-0.0404737,,-0.00688373,-0.0186959,-0.0384831,,-0.0403069,-0.00526417,-0.0112066,...,-0.0173347,0.405282,-0.0160644,-0.0113383,-0.0106003,-0.00947102,-0.00746895,-0.00751809,-0.0053622,-0.00553058
8,-0.0156676,-0.0124056,,-0.00262592,-0.0081874,-0.0097859,0.114006,-0.018518,-0.00290423,-0.0104207,...,-0.015161,-0.00319666,-0.0103068,-0.00512499,-0.0389912,-0.00760115,-0.00872094,-0.00687323,-0.00252795,-0.00478479
9,-0.00263143,-0.0423596,,-0.00543167,-0.000498482,-0.035989,,-0.0465014,-0.00406885,-0.0102674,...,-0.0186504,-0.0118204,-0.00828182,-0.0139195,-0.0134912,-0.0129906,-0.0103035,-0.00890654,-0.0049677,-0.00100779


In [19]:
# Since NMF dimensions can be interpreted as topics, let's look at the dimensions
words = tfidf_vectorizer.get_feature_names()
n_top_words = 10 # print 10 words by dimension. You can change this number

# Loop for each dimension: what words are the most dominant in each dimension
for i_dimension, dimension in enumerate(nmf.components_):
    print("Topic #%d:" % i_dimension)
    print(" ".join([words[i] for i in dimension.argsort()[:-n_top_words - 1:-1]]))
print()

# Can you interpret these dimensions as humanly intelligible topics?

Topic #0:
phone good use nice amazing really awesome got time far
Topic #1:
iphone plus new https better upgrade got android ios video
Topic #2:
s8 samsung galaxy android note better battery phones plus display
Topic #3:
apple https new ios android samsung phones people think battery
Topic #4:
great camera battery easy use works features fast life quality
Topic #5:
like really don good people better use face button think
Topic #6:
love new features camera amazing easy use size plus awesome
Topic #7:
screen size plus battery big life camera amazing nice home
Topic #8:
just don got people face know time use want make
Topic #9:
best buy far ve phones price camera good people fast



# BUILD LDA

In [20]:
from gensim import models, corpora
from gensim.utils import simple_preprocess
from gensim.models import Phrases


## attention import numpy pour le mettre à jour

In [24]:
STOPWORDS = stop_words.get_stop_words(language='en')

In [22]:
def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

# Graph-of-words

# Sentiment Analysis

In [6]:
import sys
!{sys.executable} -m pip install textblob

from textblob import TextBlob

In [7]:
text.head()

0    love my s8! awesome screen and takes great pic...
1    i mean, while i don't think that's an especial...
2    it would appear that this "open box" like new ...
3    during all ur review u just talked about getti...
4        i really want to trade my 7 plus for the s8 ?
Name: text, dtype: object

In [8]:
# Tokenize by sentences

from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

tokens = text.apply(nltk.sent_tokenize, language = 'english')

tokens[0]

['love my s8!',
 'awesome screen and takes great pictures!',
 '; love my s8!',
 'awesome screen and takes great pictures!']

In [16]:
# TextBlob

pola = [TextBlob(text).sentiment.polarity for token in tokens for text in token ]
sentences = [text for token in tokens for text in token]
df = pd.DataFrame({'pola': pola,'sentences': sentences})

df.head(n = 20)

,pola,sentences
0,0.625000,love my s8!
1,1.000000,awesome screen and takes great pictures!
2,0.625000,; love my s8!
3,1.000000,awesome screen and takes great pictures!
4,0.246875,"i mean, while i don't think that's an especial..."
5,0.500000,most people on this sub channel that emotion i...
6,0.045455,"it would appear that this ""open box"" like new ..."
7,0.565000,"i returned it, and best buy was very professio..."
8,1.000000,their customer service was excellent.
9,-0.125000,during all ur review u just talked about getti...


In [ ]:
# Next steps : select the level of polarization we wish to keep for the classification

# https://www.linkedin.com/pulse/twitter-data-mining-python-gephi-case-synthetic-biology-mikko-dufva/
# http://snap.stanford.edu/agm/
# AGM